<a href="https://colab.research.google.com/github/MengOonLee/WebScrapy/blob/master/Groceries/Lotus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --no-cache-dir -qU scrapy selenium

In [86]:
%%writefile Lotus.py
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import scrapy
from scrapy import crawler, loader
from itemloaders import processors
import logging
logging.disable("WARNING")

class LotusItem(scrapy.Item):
    category = scrapy.Field(output_processor=processors.TakeFirst())
    name = scrapy.Field(output_processor=processors.TakeFirst())
    price = scrapy.Field(output_processor=processors.TakeFirst())
    info = scrapy.Field()

class LotusSpider(scrapy.Spider):
    name = 'Lotus'

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        options = webdriver.chrome.options.Options()
        options.add_argument("--headless")
        options.add_argument("--no-sandbox")
        options.add_argument("--enable-javascript")
        options.add_argument("--enable-cookies")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-web-security")
        options.add_argument("--incognito")
        self.driver = webdriver.Chrome(options=options)

    def start_requests(self):
        urls = [
            "https://www.lotuss.com.my/en/category/meat-poultry/meat/parts-weighted"
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse_items)

    def parse_items(self, response):
        self.driver.get(response.url)

        elem_present = ""
        while not elem_present:
            try:
                elem_present = self.driver.find_element(By.XPATH,
                    "//div[@class='product-grid-item']"
                )
            except:
                continue

        selector = scrapy.Selector(text=self.driver.page_source)
        category = selector.css("h1#category-title::text").get()
        print(category)

        category_links = selector.css("div.carousel a")
        if len(category_links)!=0:
            yield from response.follow_all(category_links,
                callback=self.parse_items)

        else:
            last_height = self.driver.execute_script(
                "return document.body.scrollHeight"
            )

            while True:
                self.driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight)"
                )
                time.sleep(3)
                new_height = self.driver.execute_script(
                    "return document.body.scrollHeight"
                )
                if new_height==last_height:
                    break
                last_height = new_height

            selector = scrapy.Selector(text=self.driver.page_source)
            items = selector.css("div.product-grid-item")
            for item in items:
                l = loader.ItemLoader(item=LotusItem(), selector=item)
                l.add_css("name", "a#product-title::text")

                price = item.css("p")[0].css("::text").getall()
                l.add_value("price", "".join(price))

                img_link = item.css("img::attr(src)").get()

                # info_page = item.css("a::attr(href)").get()
                # if info_page is not None:
                #     yield response.follow(info_page,
                #         callback=self.parse_info)

                print(l.load_item())
                # return l.load_item()

            # info_links = selector.css("div#product-list a")
            # yield from response.follow_all(info_links,
            #     callback=self.parse_info)

    def parse_info(self, response):
        self.driver.get(response.url)

        selector = scrapy.Selector(text=self.driver.page_source)
        info = selector.css("div.MuiBox-root::text")[2].get()
        print(info)
        yield info


process = crawler.CrawlerProcess()
process.crawl(LotusSpider)
process.start()

Overwriting Lotus.py


In [87]:
%%bash
python Lotus.py

Parts & Weighted
{'name': "LOTUS'S FRESH AUS LAMB SHOULDER SLICE (5319)", 'price': 'RM28.97/Kg'}
{'name': 'GOURMESSA BEEF BREAKFAST STREAKY 100G', 'price': 'RM11.59'}
{'name': "LOTUS'S FRESH SLICE BEEF 300G", 'price': 'RM16.99'}
{'name': 'AUSTRALIAN NEW ZEALAND MUTTON SHOULDER (5479)', 'price': 'RM27.93/Kg'}
{'name': 'AUSTRALIA MUTTON LOIN SLICE (5318)', 'price': 'RM39.97/Kg'}
{'name': 'INDIA BUFFALO BLOCK KG (5574)', 'price': 'RM21.87/Kg'}
{'name': 'AUSTRALIA MUTTON CUBES (5317)', 'price': 'RM26.87/Kg'}
{'name': 'BUFFALO MINCE KG (5818)', 'price': 'RM19.89/Kg'}
{'name': 'LOTUSS AUSTRALIA BEEF BRISKET SHABU-SHABU 180G', 'price': 'RM14.49'}
{'name': 'LOTUSS AUSTRALIA LAMB SHABU-SHABU 180G', 'price': 'RM16.49'}


/usr/local/lib/python3.10/dist-packages/scrapy/utils/request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)
